In [9]:
import numpy as np
import pandas as pd
import datetime

In [68]:
date_list = np.array(['2005-02-01', '2005-02-02', '2005-02-03',
       '2005-02-05', '2005-02-06', '2005-02-07', '2005-02-08', '2005-02-09',
       '2005-02-11', '2005-02-12',
       '2005-02-14', '2005-02-15', '2005-02-16', '2005-02-17',
       '2005-02-19', '2005-02-20',
       '2005-02-22', '2005-02-23', '2005-02-24',
       '2005-02-25', '2005-02-26', '2005-02-27', '2005-02-28'],
      dtype='datetime64[D]')

In [69]:
ps = pd.Series(date_list)
ps

0    2005-02-01
1    2005-02-02
2    2005-02-03
3    2005-02-05
4    2005-02-06
5    2005-02-07
6    2005-02-08
7    2005-02-09
8    2005-02-11
9    2005-02-12
10   2005-02-14
11   2005-02-15
12   2005-02-16
13   2005-02-17
14   2005-02-19
15   2005-02-20
16   2005-02-22
17   2005-02-23
18   2005-02-24
19   2005-02-25
20   2005-02-26
21   2005-02-27
22   2005-02-28
dtype: datetime64[ns]

In [70]:
day1 = datetime.timedelta(days=1)
day1

datetime.timedelta(days=1)

In [71]:
grp = ps.diff().ne(day1).cumsum()
df = pd.concat([ps, grp], axis=1)
df.columns=['date', 'group']
df

,date,group
0,2005-02-01,1
1,2005-02-02,1
2,2005-02-03,1
3,2005-02-05,2
4,2005-02-06,2
5,2005-02-07,2
6,2005-02-08,2
7,2005-02-09,2
8,2005-02-11,3
9,2005-02-12,3


In [72]:
df_groupby=df.groupby('group').size()
df_groupby= df_groupby.sort_values(axis=0, ascending=False)
df_groupby

group
6    7
2    5
4    4
1    3
3    2
5    2
dtype: int64

In [73]:
maxgroup = df_groupby.index[0]
maxvalue = df_groupby[maxgroup]
dates = df[df.group==maxgroup]
print(maxgroup, maxvalue)
print(dates)

6 7
         date  group
16 2005-02-22      6
17 2005-02-23      6
18 2005-02-24      6
19 2005-02-25      6
20 2005-02-26      6
21 2005-02-27      6
22 2005-02-28      6


In [87]:
ps.size

23

In [96]:
import pandas as pd
import datetime

def longest_consecutive_dates_ps(series):
    if series.empty:
        return 0, None
    elif series.size ==1:
        return 1, series
    day1 = datetime.timedelta(days=1)
    grp = series.diff().ne(day1).cumsum()
    df = pd.concat([ps, grp], axis=1)
    df.columns=['date', 'group']
    df_groupby=df.groupby('group').size().sort_values(axis=0, ascending=False)
    maxgroup = df_groupby.index[0]
    maxvalue = df_groupby[maxgroup]
    dates = df['date'][df.group==maxgroup]
    return maxvalue, dates
    

In [97]:
longest_consecutive_dates_ps(ps)

(7,
 16   2005-02-22
 17   2005-02-23
 18   2005-02-24
 19   2005-02-25
 20   2005-02-26
 21   2005-02-27
 22   2005-02-28
 Name: date, dtype: datetime64[ns])

In [79]:
def longest_consecutive_dates1(date_list):
    if not date_list:
        return 0, None
    if len(date_list)==1:
        return 1, date_list
    
    maxstart, maxend = 0, 0
    istart = 0
    for i in range(1, len(date_list)):
        if date_list[i] - date_list[istart] != np.timedelta64(i-istart, 'D'):
            if i - istart > maxend - maxstart:
                maxstart, maxend = istart, i
            istart = i
    if i - istart > maxend - maxstart:
        maxstart, maxend = istart, i   
    # list range end is not inclusive - need to add 1 to make it inclusive:
    maxend +=1
    return len(date_list[maxstart:maxend]), date_list[maxstart:maxend]

In [80]:
longest_consecutive_dates1(list(date_list))

(7,
 [numpy.datetime64('2005-02-22'),
  numpy.datetime64('2005-02-23'),
  numpy.datetime64('2005-02-24'),
  numpy.datetime64('2005-02-25'),
  numpy.datetime64('2005-02-26'),
  numpy.datetime64('2005-02-27'),
  numpy.datetime64('2005-02-28')])

In [81]:
from datetime import timedelta, date

def longest_consecutive_dates(date_list):

    if not date_list:
        return 0, None
    if len(date_list)==1:
        return 1, date_list
    
    # store the longest interval and the current consecutive interval
    # as we iterate through a list
    max_index = this_index =  0
    max_len = this_len = 1

    # using zip here to reduce the number of indexing operations
    # this will turn the days list into [(2020-01-1, 2020-01-02), (2020-01-02, 2020-01-03), ...]
    # use enumerate to get the index of the current day
    day1 = timedelta(days=1)
    for i in range(1, len(date_list)):
        if date_list[i] - date_list[i-1] == day1:
            this_len += 1         
        else:
            if this_len > max_len:
                # we broke the record! record it as the longest interval
                max_index = this_index
                max_len = this_len               
            this_index = i
            this_len = 1            
    if this_len > max_len:
        # we broke the record! record it as the longest interval
        max_index = this_index
        max_len = this_len   

    return max_len, date_list[max_index:max_index + max_len]

In [82]:
# example input
days = [
    date(2020, 1, 1), date(2020, 1, 2), date(2020, 1, 4),
    date(2020, 1, 5), date(2020, 1, 6), date(2020, 1, 8),
]
print(longest_consecutive_dates(days))

(3, [datetime.date(2020, 1, 4), datetime.date(2020, 1, 5), datetime.date(2020, 1, 6)])


In [83]:
print(longest_consecutive_dates(list(date_list)))

(7, [numpy.datetime64('2005-02-22'), numpy.datetime64('2005-02-23'), numpy.datetime64('2005-02-24'), numpy.datetime64('2005-02-25'), numpy.datetime64('2005-02-26'), numpy.datetime64('2005-02-27'), numpy.datetime64('2005-02-28')])
